# Monitoreo Espacios __NO__ Verdes

### Idea, Secuencia y Suposiciones/Hipotesis

En este momento buscamos calcular ciertos estadisticos de las bandas de interes para cada roi de interes __que en principio no son espacios verdes__.
.
#### Suposiciones/Hipotesis

- Utilizamos solo la mascara de nubes provista por sentinel.

#### Idea

La idea consiste en caracterizar via algun estadistico el conjunto de puntos abarcado por cada parche para cada fecha disponible. El tamaño de cada parche es arbitrario asi como los estadisticos elegidos.
Esto nos va a permitir construir una serie temporal para un dado parche.

#### Secuencia

- Se procesa cada imagen y se genera un recorte sobre la ciudad en cuestion (parche)

- El parche obtenido se genera con cierto numero de bandas y se guarda.
    
- Las bandas que se utilizan se indica en el procesamiento y siempre corresponden con una resolucion de 10 m. En aquellos casos que la banda tenga menor resolucion se resamplea a 10 m.

- Una vez procesadas todas las imagenes satelitales (con la consecuente generacion de parches).Pasamos a la etapa de calculos de estadisticos sobre cada parche


In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import geopandas as gpd
import os
import glob
import matplotlib.pyplot as plt
from rasterio.plot import plotting_extent
from earthpy import plot
from pathlib import Path
import rasterio as rio
import json
import numpy as np
import pandas as pd

#
from eo_forge.io.SentinelLoaders import Sentinel2Loader

#
import sys

sys.path.append("..")

from src.ioCalc import (
    get_zonal_stats_indexes_all_geom,
    apply_mask_return_array,
    get_raster_stats,
    get_stats,
)

#
import dask
from dask.distributed import Client, progress
from dask.delayed import delayed

Los datos concernientes a Córdoba capital los extraemos del set general de datos

In [2]:
client = Client(n_workers=8, threads_per_worker=1,processes=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 62.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34421,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 62.65 GiB
Comm: tcp://127.0.0.1:34119,Total threads: 1
Dashboard: http://127.0.0.1:45477/status,Memory: 7.83 GiB
Nanny: tcp://127.0.0.1:35871,


In [3]:
gpd_cba = gpd.read_file("../data/cba/no-espacios-verdes-cba.gpkg")
gpd_cba_crs = gpd_cba.to_crs(32720)

In [4]:
gpd_cba_crs.head()

,id,desc,geometry
0,1.0,Barrio,POINT (385668.569 6527813.487)
1,1.0,Barrio,POINT (387470.788 6527423.006)
2,1.0,Barrio,POINT (391600.874 6528541.884)
3,1.0,Barrio,POINT (382687.398 6527685.830)
4,2.0,Barrio-Zona-Verde,POINT (383303.157 6527460.552)


In [5]:
gpd_cba_crs['geometry']=gpd_cba_crs.buffer(100)

In [6]:
gpd_cba_crs['area_m2']=gpd_cba_crs.area
gpd_cba_crs['osm_id']=[f"{i}-nev" for i in np.arange(gpd_cba_crs.shape[0])]
gpd_cba_crs['name']='undefined'

In [7]:
gpd_cba_crs.head()

,id,desc,geometry,area_m2,osm_id,name
0,1.0,Barrio,"POLYGON ((385768.569 6527813.487, 385768.088 6...",31365.484905,0-nev,undefined
1,1.0,Barrio,"POLYGON ((387570.788 6527423.006, 387570.307 6...",31365.484905,1-nev,undefined
2,1.0,Barrio,"POLYGON ((391700.874 6528541.884, 391700.393 6...",31365.484905,2-nev,undefined
3,1.0,Barrio,"POLYGON ((382787.398 6527685.830, 382786.917 6...",31365.484905,3-nev,undefined
4,2.0,Barrio-Zona-Verde,"POLYGON ((383403.157 6527460.552, 383402.675 6...",31365.484905,4-nev,undefined


In [9]:
gpd_cba_crs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        348 non-null    float64 
 1   desc      350 non-null    object  
 2   geometry  350 non-null    geometry
 3   area_m2   350 non-null    float64 
 4   osm_id    350 non-null    object  
 5   name      350 non-null    object  
dtypes: float64(2), geometry(1), object(3)
memory usage: 16.5+ KB


La idea en lo que sigue es obtener la imagen satelital correspondiente a la zona y generar los parametros correspondientes (sin demasiada complejidad).

In [10]:
BASE_DIR = Path("../TMP-products-int")
os.makedirs(BASE_DIR, exist_ok=True)
# make crop dir
CROP_DIR = BASE_DIR / "images-croped"
os.makedirs(CROP_DIR, exist_ok=True)
#
IMAGES_RAW = "/data/sat-images/sentinel2"

In [11]:
f"Tenemos {len(os.listdir(IMAGES_RAW))} imagenes"

'Tenemos 276 imagenes'

In [12]:
crop_images = False # ya habiamos recortado las imagenes
CROPPED_FILES_JSON='../data/cba/cropped-files.json'
CROPPED_FILES_CSV='../data/cba/cropped-files.csv'
if crop_images:
    #
    # Init class
    gpd_box = gpd.read_file("../data/cba/cordoba.geojson")
    BANDS = ["B02", "B03", "B04", "B05", "B06", "B07", "B8A", "B08", "B11", "B12"]
    sl = Sentinel2Loader(IMAGES_RAW, bands=BANDS, resolution=10, bbox=gpd_box)
    #
    delayed_crop = delayed(sl.execute)
    # make delayed task
    crop_list = []
    for i in os.listdir(IMAGES_RAW):
        crop_list.append(
            delayed_crop(
                i,
                raster_return_open=False,
                write_file="_cba",
                folder_proc_=CROP_DIR,
            )
        )
    # compute
    cropped_result = dask.compute(crop_list)
    with open(CROPPED_FILES_JSON, "w") as f:
        json.dump(cropped_result[0], f)

In [13]:
with open(CROPPED_FILES_JSON) as f:
    cropped_result = json.load(f)

In [14]:
crop_df=pd.DataFrame(cropped_result)

In [15]:
crop_df.head()

,raster,raster_cloud,match,raster_path,raster_cloud_path
0,None,None,TOTAL,../TMP-products-int/images-croped/S2B_MSIL1C_2...,../TMP-products-int/images-croped/S2B_MSIL1C_2...
1,None,None,TOTAL,../TMP-products-int/images-croped/S2B_MSIL1C_2...,../TMP-products-int/images-croped/S2B_MSIL1C_2...
2,None,None,TOTAL,../TMP-products-int/images-croped/S2A_MSIL1C_2...,../TMP-products-int/images-croped/S2A_MSIL1C_2...
3,None,None,TOTAL,../TMP-products-int/images-croped/S2B_MSIL1C_2...,../TMP-products-int/images-croped/S2B_MSIL1C_2...
4,None,None,TOTAL,../TMP-products-int/images-croped/S2A_MSIL1C_2...,../TMP-products-int/images-croped/S2A_MSIL1C_2...


In [ ]:
# Misc
STATS_BASE = BASE_DIR / "stats_calc_not_ev"
os.makedirs(STATS_BASE, exist_ok=True)
#
stats_results = []
# func
delayed_raster_stats = delayed(get_raster_stats)
#
for r_dict in cropped_result:
    raster_path = r_dict["raster_path"]
    raster_cloud_path = r_dict["raster_cloud_path"]
    stats_results.append(
        delayed_raster_stats(
            raster_path, raster_cloud_path, gpd_cba_crs, stats_base=STATS_BASE
        )
    )
#
stats_computed = dask.compute(stats_results)

/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/frojo/miniconda3/envs/monitoreo-espacios-verdes/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarn

Tenemos el conjunto de resultados para cada raster. Ahora lo aumentamos con
- la cobertura de nubes a nivel de escena
- datos relacionados a espacios verdes (identificadores,etc)

Esto lo computamos en [ab-initio-mev-cba-3](./ab-initio-mev-cba-3.ipynb), en esta situacion traemos el resultado simplemente.


In [17]:
df_clouds=pd.read_csv("../data/cba/clouds-on-raster-shape.csv")

Del geopackage que contiene los poligonos de cada espacio NO verde solo nos vamos a llevar el osm_id, name, desc, para poder efectuar los merge con alguna clave.
Entonces para cada csv con los estadisticos calculados:
- leemos el csv
- incluimos las nubes (clouds-on-raster-shape.csv)
- incluimos osm_id y name (espacios-verdes-cba.gpkg)
- calculamos la fecha y hacemos el sort correspondiente.

In [18]:
gpd_cba_crs.head()

,id,desc,geometry,area_m2,osm_id,name
0,1.0,Barrio,"POLYGON ((385768.569 6527813.487, 385768.088 6...",31365.484905,0-nev,undefined
1,1.0,Barrio,"POLYGON ((387570.788 6527423.006, 387570.307 6...",31365.484905,1-nev,undefined
2,1.0,Barrio,"POLYGON ((391700.874 6528541.884, 391700.393 6...",31365.484905,2-nev,undefined
3,1.0,Barrio,"POLYGON ((382787.398 6527685.830, 382786.917 6...",31365.484905,3-nev,undefined
4,2.0,Barrio-Zona-Verde,"POLYGON ((383403.157 6527460.552, 383402.675 6...",31365.484905,4-nev,undefined


In [19]:
#salvamos el procesado
gpd_cba_crs.to_file("../data/cba/no-espacios-verdes-proc-cba.gpkg",driver='GPKG')

In [20]:
aux_df = []
for csv in glob.glob("../TMP-products-int/stats_calc_not_ev/*.csv"):
    df_tmp = pd.read_csv(csv, index_col=0)
    df_tmp["clouds_on_raster"] = df_clouds.loc[
        df_clouds.raster == df_tmp["raster"][0], "clouds_on_shape"
    ].values[0]
    df_tmp["osm_id"] = gpd_cba_crs["osm_id"]
    df_tmp["name"] = gpd_cba_crs["name"]
    df_tmp["area_m2"] = gpd_cba_crs["area_m2"]
    df_tmp["desc"] = gpd_cba_crs["desc"]
    df_tmp["date"] = pd.to_datetime(
        df_tmp["raster"].apply(lambda x: x.split("_")[2][:8]), format="%Y%m%d"
    )
    aux_df.append(df_tmp)

In [21]:
df_cat = pd.concat(aux_df)

In [22]:
df_cat.sort_values("date", inplace=True)
df_cat.reset_index(drop=True, inplace=True)
df_cat.to_csv("../TMP-products-int/espacios-NO-verdes-indexs-cba-20170101-20220420.csv")

In [23]:
client.close()